In [ ]:
import pandas as pd
import numpy as np
import datetime
import warnings
warnings.filterwarnings('ignore')
from IPython.display import Image, HTML

import time
from datetime import datetime, timedelta

## This Notebook assumes the following files are available:

- project-to-commit (p2c) map for the hackathon projects. 
- commit-to-blob (c2b) map for the hackathon commits 
- blob-to-author (b2a) map for the hackathon blobs 
- project-to-author (p2a) map for the hackathon projects 

- DEVPOST project-to-wocURL information (woc-urls.csv) csv file 
- DEVPOST project information (projects-incl-hackathon.csv) csv file 

- Code Blobs data (prog_blobs) file 

- project-to-author (p2a) map for other projects 
- commit-to-project (c2P) map for first commits of each hackathon blob 
- author-to-author (a2A) map for the first author of each hackathon blob 


See README for corresponding commands using World of Code tool.

In [ ]:
# Load Data

HackDF = pd.read_csv('SampleData/projects-incl-hackathon.csv',sep=';')
hbacDF = pd.read_csv('SampleData/b2a.csv',sep=';', encoding = "ISO-8859-1")
hcbDF = pd.read_csv('SampleData/c2b.csv',sep=';')
hpcDF = pd.read_csv('SampleData/p2c.csv',sep=';')
wocURLDF = pd.read_csv('SampleData/woc-urls.csv',sep=',')
hpaDF = pd.read_csv('SampleData/hp2a.csv',sep=';')
p2aDF = pd.read_csv('SampleData/p2aFirst.csv', sep= ';')
c2PLinesDF = pd.read_csv('SampleData/c2PFirst.csv', sep=';')
AuthorsMergeMap = pd.read_csv('SampleData/AuthorsMergeMap', sep=';')
prog_blobs = pd.read_csv('SampleData/prog_blobs')


### Steps for TimingFlag

In [ ]:
# Prepare hackathon info
df = HackDF[['id','hackathon-id','hackathon-end-date']]
df.rename(columns = {'id':'devpostID', 'hackathon-id':'hackathonID', 'hackathon-end-date':'hackathonEndDate'}, inplace = True) 

In [ ]:
# Prepare hackathon info ( Calculate Start Date)

df['hackathonStartDate'] = df.apply(lambda row: datetime.strptime(row.hackathonEndDate, '%Y-%m-%d') - timedelta(days=2), axis=1)
HackDuration = df

In [ ]:
# df.to_csv('HackathonDuration.csv', index=False, sep=';')


In [ ]:
# Filter only Code Blobs by Github Lingustic tool

hbacDFWCommon = pd.merge(hbacDF, prog_blobs, how='inner', left_on=['BlobHash'], right_on=['Blob'])
hbacDFWCommon

In [ ]:
# Join Blobs with Hackathon Commits

BC = pd.merge(hbacDFWCommon, hcbDF, how='inner', left_on=['BlobHash'], right_on=['BlobHash'])[['BlobHash','CommitHash','FirstTimestamp','FirstAuthorID','FirstCommitHash']]
BC

In [ ]:
# Join BC dataset with Hackathon projects

BCP = pd.merge(BC, hpcDF, how='inner', left_on=['CommitHash'], right_on=['CommitHash'])[['BlobHash','ProjectID','FirstTimestamp','FirstAuthorID','FirstCommitHash']].drop_duplicates()
BCP

In [ ]:
# Join with wocURLs

BCP_DevPost = pd.merge(BCP, wocURLDF, how='inner', left_on=['ProjectID'], right_on=['ProjectID'])[['devpost_id','hackathon_id','ProjectID','BlobHash','FirstTimestamp','FirstAuthorID','FirstCommitHash']].drop_duplicates()

BCP_DevPost

In [ ]:
%%time
# Join with hackathon durations
compareDF = pd.merge(BCP_DevPost, HackDuration, how='inner', left_on=['devpost_id','hackathon_id'], right_on = ['devpostID','hackathonID'])[['devpost_id','hackathon_id','ProjectID','BlobHash','FirstTimestamp','FirstAuthorID','FirstCommitHash','hackathonStartDate','hackathonEndDate']].drop_duplicates()
compareDF

In [ ]:
# Fix timestamps for start/end dates

def addMaxTime(dcol):
    return str(dcol) + ' 23:59:59'
    
def addMinTime(dcol):
    return str(dcol) + ' 00:00:00'

compareDF['hackathonEndDate'] = compareDF['hackathonEndDate'].apply(addMaxTime)

compareDF['hackathonStartDate'] =  pd.to_datetime(compareDF['hackathonStartDate'], format='%Y-%m-%d %H:%M:%S')
compareDF['hackathonEndDate'] =  pd.to_datetime(compareDF['hackathonEndDate'], format='%Y-%m-%d %H:%M:%S')
# compareDF.info()
compareDF

In [ ]:
# Calculate TimingFlag

%%time

def compareDates(row):
    hackathonStartDate = datetime.strptime(str(row.hackathonStartDate), '%Y-%m-%d %H:%M:%S')
    hackathonEndDate = datetime.strptime(str(row.hackathonEndDate), '%Y-%m-%d %H:%M:%S')
    FirstTimestamp = datetime.strptime(str(row.FirstTimestamp), '%Y-%m-%d %H:%M:%S')
    if (hackathonStartDate < FirstTimestamp < hackathonEndDate):
        return 2            # Between
    elif (FirstTimestamp > hackathonEndDate):
        return 3            # After
    else:
        return 1            # Before
    
def iterrows_impl(df):
    return pd.Series(
        compareDates(row)     
        for row in df.itertuples()
    )
  

compareDF['TimingFlag'] = pd.Series(iterrows_impl(compareDF))

In [ ]:
# Grouping for results (Count per TimingFlag)

result1 = compareDF[['BlobHash','TimingFlag']].groupby(['TimingFlag']).agg(['count'])
result1['Percentage'] = result1.apply(lambda x: 100 * x / float(x.sum()))
result1 = result1.reset_index()
result1.columns = ['TimingFlag','BlobHash','Percentage']
result1
# If Before then TimingFlag = 1
# If between then TimingFlag = 2
# If After then TimingFlag = 3

In [ ]:
from matplotlib import pyplot as plt 
import numpy as np 
from matplotlib.gridspec import GridSpec

plt.figure(1, figsize=(20,20)) 
cmap = plt.get_cmap('Spectral')
colors = [cmap(i) for i in np.linspace(0, 1, 8)]

# Creating dataset 
Labels = result1[['TimingFlag']].values #['Before', 'During', 'After'] 
data = result1[['BlobHash']].values.flatten() 
perc = result1[['Percentage']].values
  
# Creating plot 
the_grid = GridSpec(2, 2)
plt.subplot(the_grid[0, 1], aspect=1, title='Percentage based on Timing Flag')
plt.pie(data, labels = Labels, autopct='%1.1f%%', shadow=True, colors=colors) 
  
# show plot 
plt.show() 

In [ ]:
from matplotlib.pyplot import figure
figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')

x = result1.sort_values(by=['Percentage'])[['TimingFlag']].values #['During','After', 'Before']
data = result1.sort_values(by=['Percentage'])[['Percentage']].values.flatten() 

x_pos = [i for i, _ in enumerate(x)]

plt.barh(x_pos, data, color='green')
plt.ylabel("Blob generation category")
plt.xlabel("Percentage")
plt.title("Percentage based on Timing Flag")

plt.yticks(x_pos, x)

for i, v in enumerate(data):
    plt.text(v +0.5, i -0.1, str(round(v,2)) + '%', color='Black')

plt.show()

In [ ]:
from matplotlib.pyplot import figure
figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')

x = result1.sort_values(by=['Percentage'])[['TimingFlag']].values #['During','After', 'Before']
data = result1.sort_values(by=['Percentage'])[['BlobHash']].values.flatten() 

x_pos = [i for i, _ in enumerate(x)]

plt.barh(x_pos, data, color='green')
plt.ylabel("Blob generation category")
plt.xlabel("Count")
plt.title("Blob Count based on Timing Flag")

plt.yticks(x_pos, x)

for i, v in enumerate(data):
    plt.text(v +0.5, i -0.1, str(round(v,2)) , color='Black')

plt.show()

### Steps for AuthorFlag

In [ ]:
p2aDF = p2aDF.drop_duplicates()
p2aDF

In [ ]:
c2PLinesDF = c2PLinesDF.drop_duplicates()
c2PLinesDF

In [ ]:
# Join to add hackathon project authors
%%time

compareDF2 = pd.merge(compareDF, hpaDF, how='inner', left_on=['ProjectID'], right_on=['ProjectID'])
compareDF2

In [ ]:
# Merge author name aliases
compareDF22 = pd.merge(compareDF2, AuthorsMergeMap,how='left', left_on=['FirstAuthorID'], right_on=['AuthorsMap'])
compareDF22['FirstAuthorL'] = compareDF22['FirstAuthorID'] +','+ compareDF22['AuthorAlias'].fillna('')
del compareDF22['AuthorsMap']
del compareDF22['AuthorAlias']
compareDF22

In [ ]:
# Join to get projects for each initial commit of a blob
compareDF3 = pd.merge(compareDF22, c2PLinesDF, how='inner', left_on=['FirstCommitHash'], right_on = ['CommitHash'])
compareDF3

In [ ]:
# Join to get the authors for each project
compareDF3 = pd.merge(compareDF3, p2aDF, how='inner', left_on=['CProject'], right_on = ['ProjectID'])
compareDF3

In [ ]:
# Logic to get AuthorFlag
%%time
        
def checkAuthor(row):
    AuthorL = str(row.AuthorL).split(",")        # Hackathon Team
    AuthorL = list(filter(None, AuthorL))
    PAuthorL = str(row.PAuthorL).split(",")      # Commit Authors from other projects
    PAuthorL = list(filter(None, PAuthorL))
    FirstAuthorL = str(row.FirstAuthorID).split(",")
    FirstAuthorL = list(filter(None, FirstAuthorL))
    HAuthInteract = list(set(FirstAuthorL) & set(AuthorL))
    if (len(HAuthInteract) > 0):
        return 1
    else:
        PAuthorIntersect = list(set(AuthorL) & set(PAuthorL))
        if(len(PAuthorIntersect)) > 0:
            return 2
        else:
            return 3

    
def iterrows_impl(df):
    return pd.Series(
        checkAuthor(row)     
        for row in df.itertuples()
    )
  

compareDF3['AuthorFlag'] = pd.Series(iterrows_impl(compareDF3))


In [ ]:
# Save Dataframe
compareDF3[['devpost_id','devpost_id','ProjectID_x','hackathonStartDate','hackathonEndDate','BlobHash','TimingFlag','AuthorFlag']].to_csv('compareDF3_20210104.csv',sep=';')


In [ ]:
# Change AuthorFlag to string
compareDF3['AuthorFlag']= compareDF3['AuthorFlag'].apply(str)

In [ ]:
# Concatenate AuthorFlag , since a commit can be used in many projects in c2P
finalResultDF = compareDF3[['devpost_id','ProjectID_x','hackathonStartDate','hackathonEndDate','BlobHash','TimingFlag','AuthorFlag']].groupby(['devpost_id','ProjectID_x','hackathonStartDate','hackathonEndDate','BlobHash','TimingFlag']).agg({'AuthorFlag':lambda x : ','.join(set(x))}).reset_index()
finalResultDF

In [ ]:
# logic to get AuthorFlagMin

def checkUsageMin(row):
    AuthorFlagL = str(row.AuthorFlag).split(",")        
    AuthorFlagMin = min(AuthorFlagL)
    return AuthorFlagMin


    
def iterrows_impl(df):
    return pd.Series(
        checkUsageMin(row)     
        for row in df.itertuples()
    )
  

finalResultDF['AuthorFlagMin'] = pd.Series(iterrows_impl(finalResultDF))

finalResultDF

In [ ]:
# Rename columns
del finalResultDF['AuthorFlag']
finalResultDF.rename(columns={'AuthorFlagMin':'AuthorFlag'}, inplace=True)
finalResultDF

In [ ]:
# Calculate count for each AuthorFlag

result2 = finalResultDF[['BlobHash','AuthorFlag']].groupby(['AuthorFlag']).agg(['count'])

result2['Percentage'] = result2.apply(lambda x: 100 * x / float(x.sum()))
result2 = result2.reset_index()
result2.columns = ['AuthorFlag','BlobHash','Percentage']
result2

# AuthorFlag = 1   Author is part of the team
# AuthorFlag = 2   Author Overlap with the origional project
# AuthorFlag = 3   Others



In [ ]:
from matplotlib import pyplot as plt 
import numpy as np 
from matplotlib.gridspec import GridSpec

plt.figure(1, figsize=(20,20)) 
cmap = plt.get_cmap('Spectral')
colors = [cmap(i) for i in np.linspace(0, 1, 8)]

# Creating dataset 
Labels = ['Same Author', 'CoAuthor', 'Other Author'] 
data = result2[['BlobHash']].values.flatten() 
  
# Creating plot 
the_grid = GridSpec(2, 2)
plt.subplot(the_grid[0, 1], aspect=1, title='Percentage based on Author Flag')
plt.pie(data, labels = Labels, autopct='%1.1f%%', shadow=True, colors=colors) 
  
# show plot 
plt.show() 

In [ ]:

from matplotlib.pyplot import figure
figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')

x = result2.sort_values(by=['Percentage'])[['AuthorFlag']].values #['CoAuthor', 'Other Author', 'Same Author'] 
data = result2.sort_values(by=['Percentage'])[['Percentage']].values.flatten() 

x_pos = [i for i, _ in enumerate(x)]

plt.barh(x_pos, data, color='black')
plt.ylabel("Blob generation category")
plt.xlabel("Percentage")
plt.title("Percentage based on Author Flag")

plt.yticks(x_pos, x)

for i, v in enumerate(data):
    plt.text(v +0.5, i -0.1, str(round(v,2)) + '%', color='Black')

plt.show()


In [ ]:
finalResultDF.to_csv('finalResultDF_20210104.csv', sep=';')


### Pivot table for each hackathon project

In [ ]:
# Groupby projectID, TimingFlag
AnalysisTF = finalResultDF.groupby(['ProjectID_x','TimingFlag']).agg({'BlobHash':'count'}).reset_index()
AnalysisTF

In [ ]:
# Create Pivot Table 
AnalysisTFPV = pd.pivot_table(AnalysisTF,index=['ProjectID_x'],values=['BlobHash'],columns=['TimingFlag'],aggfunc=[np.sum],fill_value=0).reset_index()
AnalysisTFPV.columns = ['ProjectID','Before','During','After']

AnalysisTFPV.to_csv('Analysis_TF_Pivot.csv', sep=';',index=False)
AnalysisTFPV

### Pivot for groupby TimingFlag and AuthorFlag

In [ ]:
dd = finalResultDF[['TimingFlag','AuthorFlag','BlobHash']].groupby(['TimingFlag','AuthorFlag']).count().reset_index()
dd.columns = ['TimingFlag','AuthorFlag','BlobCount']
dd['TotalBlobs'] = 6511360  # Total records in the finalResultDF dataframe
dd

In [ ]:
dd['TimingFlag'].replace({1: "Before", 2: "During", 3: "After"}, inplace=True)
dd['AuthorFlag'].replace({"1": "Same Author", "2": "Co-Author", "3": "Other Author"}, inplace=True)
dd

In [ ]:
dd['Ratio'] = round(100*(dd['BlobCount'] / dd['TotalBlobs']),2)
dd

In [ ]:
ddPV = pd.pivot_table(dd,index=['TimingFlag'],values=['Ratio'],columns=['AuthorFlag'],aggfunc=[np.sum],fill_value=0).reset_index()
ddPV.columns = ['TimingFlag','Co-Author','Other Author','Same Author']
ddPV.to_csv('RQ1Pivot.csv', sep=';', index = False)